In [8]:
# coding: utf-8
import pandas
import requests
import csv
import json
import xmltodict
from collections import OrderedDict
from pymongo import MongoClient

client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
db= client.country_info
collection = db.country_info

url = "http://apis.data.go.kr/1262000/CountryBasicService/getCountryBasicList"
sKey = "24C%2FxddyRCTiVMSQN7xGhrvODlbKHfrGG%2Bg4ryyKXO8GGVjexQKYpkSH7PHU6MamZZ9qa07Dq9h25bAohPX3Jg%3D%3D"

code_list = []
with open("isoCode.csv", 'r') as f:
    csvreader = csv.reader(f)
    for row in csvreader:
        code = row[0].split(',')
        code_list.append(code[0])

code_list = code_list[2:]



for iso in code_list:
    payload = {'serviceKey': sKey ,'isoCode1':iso}
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())

    resp = requests.get(url,params = payload_str)
    xmlString = resp.text     
    dt1 = xmltodict.parse(xmlString,encoding= 'utf-8')
    if dt1.get('response'):
        if dt1['response']['body']['items']:
            dt2 = dt1['response']['body']['items']['item']
            info = dt2['basic'][7:-6]
            infoes = info.split('ㅇ ')
            for data in infoes:
                data = data.strip()
                if '<br>' in data:
                    data = data[:-4]
                data = data.split(':')
                if len(data) is 2:
                    dt2[data[0]]=data[1]
            del dt2['basic']
    
            jsonString = json.dumps(dt2, indent=2, ensure_ascii=False)
            collection.insert_one(dt2)
 
            with open("country_info.txt", 'a',encoding='utf-8') as f:
                f.write(jsonString)


In [ ]:
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
db= client.country_info
collection = db.country_info
result = collection.insert(jsonString)
